In [1]:
import numpy as np
import os
import pretty_midi
import data

import keras
from keras import layers

Using TensorFlow backend.
/home/randy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/randy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/randy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/randy/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
mdg = data.MidiDataGenerator("clean_midi", 500)
g = mdg.flow(50)

In [3]:
batches = []
for i in range(20):
    batches.append(next(g))
    print("batches so far: {}".format(len(batches)))
#batches = np.concatenate(batches)

len:1349


/home/randy/.local/lib/python3.6/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


len:753
len:1716
len:626
len:2558
len:613
error on clean_midi/Nirvana/Polly.1.mid data byte must be in range 0..127
len:1386
error on clean_midi/Bush/Glycerine.1.mid Could not decode key with 1 flats and mode 255
len:826
len:1268
error on clean_midi/Rascel/Arrividerci Roma.mid data byte must be in range 0..127
error on clean_midi/Celine Dion/Where Does My Heart Beat Now.mid Could not decode key with 2 flats and mode 255
len:2862
len:3926
len:1850
error on clean_midi/Billy Joel/Pressure.mid data byte must be in range 0..127
len:1319
error on clean_midi/Jimi Hendrix/Purple Haze.2.mid data byte must be in range 0..127
len:687
error on clean_midi/Rednex/The Ultimate Rednex Mega Mix, Part 3: Cotton Eye Joe.mid data byte must be in range 0..127
len:3244
error on clean_midi/Clayderman Richard/Ballade Pour Adeline.mid data byte must be in range 0..127
error on clean_midi/Jennifer Lopez/If You Had My Love.mid MThd not found. Probably not a MIDI file
len:1190
len:1235
len:1403
error on clean_mid

In [26]:
x_train_octaves = []
x_train_notes = []
x_train_velocities = []
x_train_durations = []
x_train_rests = []

for batch in batches:
    x_train_octaves.extend(batch[0])
    x_train_notes.extend(batch[1])
    x_train_velocities.extend(batch[2])
    x_train_durations.extend(batch[3])
    x_train_rests.extend(batch[4])

x_train_octaves = np.array(x_train_octaves)
x_train_notes = np.array(x_train_notes)
x_train_velocities = np.array(x_train_velocities)
x_train_durations = np.array(x_train_durations)
x_train_rests = np.array(x_train_rests)

In [30]:
def create_model(stateful=False, batch_size=None, seq_len=None):
    return_sequences = not stateful
    
    reg = keras.regularizers.l2(1e-6)
    
    if not stateful:
        x_input_octaves = layers.Input((None, 8))
        x_input_notes = layers.Input((None, 12))
        x_input_velocities = layers.Input((None, 1))
        x_input_durations = layers.Input((None, 1))
        x_input_rests = layers.Input((None, 1))
    else:
        x_input_octaves = layers.Input(batch_shape=(batch_size, 1, 8))
        x_input_notes = layers.Input(batch_shape=(batch_size, 1, 12))
        x_input_velocities = layers.Input(batch_shape=(batch_size, 1, 1))
        x_input_durations = layers.Input(batch_shape=(batch_size, 1, 1))
        x_input_rests = layers.Input(batch_shape=(batch_size, 1, 1))
    
    x_input_velocities_bn = layers.BatchNormalization()(x_input_velocities)
    x_input_durations_bn = layers.BatchNormalization()(x_input_durations)
    x_input_rests_bn = layers.BatchNormalization()(x_input_rests)
    
    
    x = layers.Concatenate(2)([x_input_octaves, x_input_notes, x_input_velocities_bn, x_input_durations_bn, x_input_rests_bn])
    
    x = layers.CuDNNLSTM(256, unit_forget_bias=True, kernel_regularizer=reg, recurrent_regularizer=reg,
                         activity_regularizer=reg, return_sequences=True, stateful=stateful)(x)
    x = layers.CuDNNLSTM(256, unit_forget_bias=True, kernel_regularizer=reg, recurrent_regularizer=reg,
                         activity_regularizer=reg, return_sequences=True, stateful=stateful)(x)
    x = layers.CuDNNLSTM(256, unit_forget_bias=True, kernel_regularizer=reg, recurrent_regularizer=reg,
                         activity_regularizer=reg, return_sequences=True, stateful=stateful)(x)
    
    x_octaves = layers.TimeDistributed(layers.Dense(8, kernel_regularizer=reg))(x)
    x_notes = layers.TimeDistributed(layers.Dense(12, kernel_regularizer=reg))(x)
    x_velocities = layers.TimeDistributed(layers.Dense(1, kernel_regularizer=reg))(x)
    x_durations = layers.TimeDistributed(layers.Dense(1, kernel_regularizer=reg))(x)
    x_rests = layers.TimeDistributed(layers.Dense(1, kernel_regularizer=reg))(x)

    x_octaves = layers.Softmax(axis=2)(x_octaves)
    x_notes = layers.Softmax(axis=2)(x_notes)
    
    def cut_last(x):
        return x[:, :-1]
    
    if not stateful:
        x_octaves = layers.Lambda(cut_last, name="octaves_out")(x_octaves)
        x_notes = layers.Lambda(cut_last, name="notes_out")(x_notes)
        x_velocities = layers.Lambda(cut_last, name="velocities_out")(x_velocities)
        x_durations = layers.Lambda(cut_last, name="durations_out")(x_durations)
        x_rests = layers.Lambda(cut_last, name="rests_out")(x_rests)
    
    model = keras.models.Model([x_input_octaves, x_input_notes, x_input_velocities, x_input_durations, x_input_rests],
                               [x_octaves, x_notes, x_velocities, x_durations, x_rests])
    return model

In [33]:
model = create_model()
model.summary()

opt = keras.optimizers.Adam(1e-3)
model.compile(opt, ['categorical_crossentropy', 'categorical_crossentropy', 'mse', 'mse', 'mse'],
              {'octaves_out': 'acc', 'notes_out': 'acc'},
             loss_weights=[1, 1, 5e-4, 5, 5])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, None, 1)      0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, None, 1)      0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, None, 1)      0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, None, 8)      0                                            
__________________________________________________________________________________________________
input_22 (

In [34]:
model.fit(x=[x_train_octaves, x_train_notes, x_train_velocities, x_train_durations, x_train_rests],
          y=[x_train_octaves[:, 1:], x_train_notes[:, 1:], x_train_velocities[:, 1:], x_train_durations[:, 1:], x_train_rests[:, 1:]],
          epochs=1000, validation_split=0.2)

800/800 [==============================] - 5s 6ms/step - loss: 0.5561 - octaves_out_loss: 0.0345 - notes_out_loss: 0.0590 - velocities_out_loss: 151.0970 - durations_out_loss: 0.0083 - rests_out_loss: 0.0127 - octaves_out_acc: 0.9721 - notes_out_acc: 0.9678 - val_loss: 0.5191 - val_octaves_out_loss: 0.0240 - val_notes_out_loss: 0.0419 - val_velocities_out_loss: 153.4348 - val_durations_out_loss: 0.0076 - val_rests_out_loss: 0.0125 - val_octaves_out_acc: 0.9738 - val_notes_out_acc: 0.9715
Epoch 541/1000
800/800 [==============================] - 5s 6ms/step - loss: 0.5497 - octaves_out_loss: 0.0334 - notes_out_loss: 0.0585 - velocities_out_loss: 151.1304 - durations_out_loss: 0.0076 - rests_out_loss: 0.0126 - octaves_out_acc: 0.9730 - notes_out_acc: 0.9686 - val_loss: 0.5144 - val_octaves_out_loss: 0.0236 - val_notes_out_loss: 0.0435 - val_velocities_out_loss: 152.6815 - val_durations_out_loss: 0.0071 - val_rests_out_loss: 0.0123 - val_octaves_out_acc: 0.9740 - val_notes_out_acc: 0.9717

KeyboardInterrupt: 

In [22]:
model = keras.models.load_model('first.h5')

In [35]:
predict_model = create_model(True, 1)
predict_model.set_weights(model.get_weights())

In [40]:
sample = False

song = [[4, 6, 50, 0.25, 0]]

for i in range(500):
    last_out = [np.zeros((1, 1, 8)), np.zeros((1, 1, 12)), np.zeros((1, 1, 1)), np.zeros((1, 1, 1)), np.zeros((1, 1, 1))]
    last_out[0][0, 0, song[-1][0]] = 1
    last_out[1][0, 0, song[-1][1]] = 1
    last_out[2][0, 0, 0] = song[-1][2]
    last_out[3][0, 0, 0] = song[-1][3]
    last_out[4][0, 0, 0] = song[-1][4]
    
    output = predict_model.predict(last_out)
    
    octaves = output[0][0, 0]
    notes = output[1][0, 0]
    
    octave_selection = np.random.random()
    note_selection = np.random.random()
    
    
    if sample:
        octave = len(octaves) - 1
        note = len(notes) - 1

        for i, p in enumerate(octaves):
            octave_selection -= p
            if octave_selection <= 0:
                octave = i
                break

        for i, p in enumerate(notes):
            note_selection -= p
            if note_selection <= 0:
                note = i
                break
    else:
        octave = np.argmax(octaves)
        note = np.argmax(notes)
    
    velocity = int(output[2][0, 0, 0])
    duration = output[3][0, 0, 0]
    rest = output[4][0, 0, 0]
    
    song.append([octave, note, velocity, duration, rest])
predict_model.reset_states()

In [41]:
midi = pretty_midi.PrettyMIDI()
instrument = pretty_midi.Instrument(program=0)

time = 0

for i, (octave, note, velocity, duration, rest) in enumerate(song):
    pm_note = pretty_midi.Note(velocity, octave * 12 + note + 21, time, time + duration)
    time += duration + rest
    instrument.notes.append(pm_note)

midi.instruments.append(instrument)
midi.write('out.mid')